## Google Cloud APIs

### Audio

In [1]:
from google.cloud import speech_v1
from google.cloud.speech_v1 import enums
import io
import os
from scipy.io import wavfile
import nltk

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="D:/NYU_MS/Sem3/CloudML/key.json"

directory = 'D:/NYU_MS/Sem3/CloudML/Adveserial-Attacks/audio-dataset/audio'

client = speech_v1.SpeechClient()
# The language of the supplied audio
language_code = "en-US"

t=[]
for filename in os.listdir(directory):
    if filename.endswith(".wav"): 
        local_file_path = (os.path.join(directory, filename))
        sample_rate_hertz, content= wavfile.read(local_file_path)
        # Sample rate in Hertz of the audio data sent
        with io.open(local_file_path, "rb") as f:
            content = f.read()
        audio = {"content": content}
        config = {
            "language_code": language_code,
            "sample_rate_hertz": sample_rate_hertz
        }
        x=''
        response = client.recognize(config, audio)
        for result in response.results:
            # First alternative is the most probable result
            for tr in result.alternatives:
                x= x+tr.transcript
        t.append(x)
        

In [6]:
i=0
for ti in t:
    path = ''
    if i<10:
        path = 'D:/NYU_MS/Sem3/CloudML/Adveserial-Attacks/google_transcript/'+'0'+str(i)+'.txt'
    else:
        path = 'D:/NYU_MS/Sem3/CloudML/Adveserial-Attacks/google_transcript/'+str(i)+'.txt'
    i+=1
    file = open(path,"w") 
    file.write(ti)
    file.close() 

In [12]:
directory = 'D:/NYU_MS/Sem3/CloudML/Adveserial-Attacks/audio-dataset/script'
i=0
blues=[]
google_trans = []
ground_truth = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"): 
        local_file_path = (os.path.join(directory, filename))
        with io.open(local_file_path, "rb") as f:
            content = f.read()
            ground_truth.append(content.lower())
            
directory = 'D:/NYU_MS/Sem3/CloudML/Adveserial-Attacks/google_transcript'
i=0
for filename in os.listdir(directory):
    if filename.endswith(".txt"): 
        local_file_path = (os.path.join(directory, filename))
        with io.open(local_file_path, "rb") as f:
            content = f.read()
            google_trans.append(content.lower())

In [16]:
blues=[]
for i in range(len(google_trans)):
    google = google_trans[i].decode("utf-8").split()
    truth = ground_truth[i].decode("utf-8").split()
    #print('\nGoogle : ', google, '\nTruth : ', truth)
    #there may be several references
    blues.append(nltk.translate.bleu_score.sentence_bleu([truth], google))
    #print("\nB : ", blues[i])

In [18]:
max(blues)

0.7791187640327726

In [19]:
min(blues)

1.529588101626149e-79